In [2]:
#현재 시간 설정
from datetime import datetime, date, time, timedelta

now = datetime.now()
nowDatetime = now.strftime('%Y-%m-%d %H:%M')
print(nowDatetime)

2021-08-27 23:28


In [3]:
########### 이슈 빼오기 ########
import requests
from bs4 import BeautifulSoup

url = 'https://www.issuelink.co.kr/community/listview/all/3/adj/_self/blank/blank/blank'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36'}
html_rank = requests.get(url, headers=headers).text
soup_rank = BeautifulSoup(html_rank, 'lxml')
# keyword = soup_rank.select('div > div:nth-of-type(2) > div:nth-of-type(4) > div.ibox.float-e-margins > div ')
keywords = soup_rank.select('div.ibox.float-e-margins > div > table > tbody > tr > td > a')


key_list = []
for k in keywords:
    keyword = k.text
    key_list.append(keyword)

###### 7시를 기준으로 기준 리스트를 하나 만들어야 함 / datetime에서 시간 분만 빼와서 if로 비교
##### 7시 기준이 아닐 때 

key_list.pop(0)
print(key_list)

['호날두', '일본', '아프간', '한국', '펜트하우스', '이재명', '윤희숙', '법무부', '플립', '김용호', '도깨비', '블리치', '로아', '윤석열', '홍준표', '보라', '이야기', '코로나', '미국', '레전드', '아이돌', '이낙연', '아파트', '민주당', '유튜브', '중국', '갤럭시', '정윤종', '마지막']


In [4]:
### 이슈 실제 조회수 ###

from bs4 import BeautifulSoup
import requests
import urllib
import operator

sum_list = []
search_list = key_list

def sum_hit(search) :
    
    url = f'https://www.issuelink.co.kr/community/listview/read/3/adj/_self/blank/{search}'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36'}
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')

    hits = soup.select('span.hit')
    
    sum=0
    
    for hit in hits :
        sum += int(hit.text.replace(',',''))
    
    sum_list.append(sum)  
    print('*',end='')
    
for search in search_list :
    sum_hit(search)

#조회수, 키워드 합치기
sum_search = dict(zip(key_list,sum_list))

#조회수 순으로 정렬
#a = sorted(sum_search.items(), key=lambda x:x[1], reverse = True)

print()
print(sum_search)


*****************************
{'호날두': 135071, '일본': 126069, '아프간': 124348, '한국': 128719, '펜트하우스': 12452, '이재명': 25445, '윤희숙': 31888, '법무부': 36026, '플립': 22204, '김용호': 32121, '도깨비': 54311, '블리치': 31913, '로아': 2143, '윤석열': 8096, '홍준표': 6027, '보라': 5639, '이야기': 10234, '코로나': 31626, '미국': 24386, '레전드': 43250, '아이돌': 32196, '이낙연': 12644, '아파트': 29632, '민주당': 6260, '유튜브': 14252, '중국': 78382, '갤럭시': 20462, '정윤종': 2798, '마지막': 15996}


In [5]:
#### 키워드 제목 추출 탐30 개별로 ####

from bs4 import BeautifulSoup
import requests
import urllib
import os

#디렉토리 폴더 생성
path = "C:/work/python/Asia_GAN/myproject/temp" #C:\work\python\Asia_GAN\myproject
if not os.path.isdir(path):                                                           
    os.mkdir(path)

keyword_list=[]

def subject(search) :
    url = f'https://www.issuelink.co.kr/community/listview/all/3/adj/_self/blank/{search}'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36'}
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')

    sub = soup.select('span.title')
    
    keyword_list.clear()
    
    for i in sub :
        split_string = i.get_text().split(' [',1)
        substring = split_string[0]    
        keyword_list.append(substring)
        
    with open(f'C:/work/python/Asia_GAN/myproject/temp/{search}.txt','w', encoding = 'utf-8') as file :
        file.writelines(keyword_list)
    
    print('**', end="")
            
    
for search in search_list :
    subject(search)

print()
print('완료')

**********************************************************
완료


In [6]:
#### 키워드 형태소 카운팅 ####

""" 형태소 분석기
    명사 추출 및 빈도수 체크
    python [모듈 이름] [텍스트 파일명.txt] [결과파일명.txt]
"""

import sys
from konlpy.tag import Twitter
from collections import Counter


def get_tags(text, ntags=50):
    spliter = Twitter()
    nouns = spliter.nouns(text)
    count = Counter(nouns)
    return_list = []
    for n, c in count.most_common(ntags):
        temp = {'tag': n, 'count': c}
        return_list.append(temp)
    return return_list


def main(search):
    # 분석할 파일
    noun_count = 50
    # count.txt 에 저장
    open_text_file = open(f'C:/work/python/Asia_GAN/myproject/temp/{search}.txt', 'r',-1,"utf-8")
    # 분석할 파일을 open 
    text = open_text_file.read() #파일을 읽습니다.
    tags = get_tags(text, noun_count) # get_tags 함수 실행
    open_text_file.close()   #파일 close
    open_output_file = open(f"C:/work/python/Asia_GAN/myproject/temp/{search}-count.txt", 'w',-1,"utf-8")
    # 결과로 쓰일 count.txt 열기
    for tag in tags:
        noun = tag['tag']
        count = tag['count']
        open_output_file.write('{} {}\n'.format(noun, count))
    # 결과 저장
    open_output_file.close() 

for search in search_list :
    main(search)
    
print('완료')


C:\Users\kim\.conda\envs\exam_cv2\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


완료


In [7]:
import requests
from bs4 import BeautifulSoup

sub_key = {}

for i in sum_search :
      
    with open(f'C:/work/python/Asia_GAN/myproject/temp/{i}-count.txt','r', encoding = 'utf-8') as file :
        data = str(file.readlines()[1])

    split_string = data.split(' ',1) 
    substring = split_string[0]           #빈도수 제거 
    #print(substring)
    
    sub_key[i] = substring
    
    
print(sub_key)

{'호날두': '맨유', '일본': '아프간', '아프간': '작전', '한국': '아프간', '펜트하우스': '단태', '이재명': '사생활', '윤희숙': '숙', '법무부': '차관', '플립': '갤럭시', '김용호': '방송', '도깨비': '게임', '블리치': '만해', '로아': '좀', '윤석열': '이재명', '홍준표': '이유', '보라': '김보라', '이야기': '지금', '코로나': '검사', '미국': '아프간', '레전드': '기사', '아이돌': '그룹', '이낙연': '이재명', '아파트': '공급', '민주당': '지지자', '유튜브': '김용호', '중국': '보고', '갤럭시': '폴드', '정윤종': '김윤환', '마지막': '김용호'}


In [8]:
#네이버 검색, 키워드 서브키워드 
import requests
from bs4 import BeautifulSoup


art_lists = []

def search(key, b) :
    
    art_list = [b]
    
    #url = f'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query={key}'
    url = f'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={key}'
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'lxml')
    
    for i in range(2) :
        news = soup.select('div.info_group > a:nth-of-type(2)')[i].attrs["href"]
        art_list.append(news)
    
    art_lists.append(art_list)
    
a = sorted(sum_search.items(), key=lambda x:x[1], reverse = True) #value 값 기준으로 정렬, 상위 5개 키워드


for i in range(5) :
    b= a[i][0]     #정렬 후 dic -> list 함수로 변환돼서 [i][0]으로 빼옴 
    #print(b)
    
    with open(f'C:/work/python/Asia_GAN/myproject/temp/{b}-count.txt','r', encoding = 'utf-8') as file :
        data = str(file.readlines()[1])

    split_string = data.split(' ',1) 
    substring = split_string[0]           #빈도수 제거 
    #print(substring)
    
    key = b + " " + substring
    search(key, b)
    
print(art_lists)
print(sum_search.items())


[['호날두', 'https://sports.news.naver.com/news.nhn?oid=076&aid=0003771921', 'https://sports.news.naver.com/news.nhn?oid=108&aid=0002983994'], ['한국', 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=104&oid=001&aid=0012625367', 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=100&oid=366&aid=0000757380'], ['일본', 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=104&oid=003&aid=0010687901', 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=104&oid=417&aid=0000729372'], ['아프간', 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=104&oid=022&aid=0003614026', 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=100&oid=018&aid=0005021393'], ['중국', 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=104&oid=003&aid=0010685077', 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=104&oid=056&aid=0011108320']]
dict_items([('호날두', 135071), ('일본', 126069), ('아프간', 124348), ('한국', 128719), ('펜트하우스', 12452), ('이

In [9]:
#subject 디렉토리내 파일 삭제하기

import shutil

pathTest = r"C:/work/python/Asia_GAN/myproject/temp"

try:
    shutil.rmtree(pathTest)
except OSError as e:
    print(e)
else:
    print("The directory is deleted successfully")

The directory is deleted successfully


In [10]:
#### 새로운 파일 생성 ###

import os
from datetime import datetime, date, time, timedelta
import datetime
import openpyxl
import pickle

#디렉토리 폴더 생성
path = "C:/work/python/Asia_GAN/myproject/output"
if not os.path.isdir(path):                                                           
    os.mkdir(path)


today =datetime.datetime.now()

hms = today.strftime('%H:%M:%S') #시분초
ymd = today.strftime('%Y-%m-%d') #년월일  - 오늘 날짜
yesterday = (today - datetime.timedelta(1)).strftime('%Y-%m-%d') #어제날짜

wb = openpyxl.Workbook()


if hms < '19:00:00':
    file_name = yesterday
    path2 = f"C:/work/python/Asia_GAN/myproject/output/{file_name}.xlsx"
    
    if not os.path.isfile(path2):  
        wb.save(path2)
        print('파일 생성완료1')

    else :
        print('파일을 작업중입니다....')

else : 
    file_name = ymd
    path2 = f"C:/work/python/Asia_GAN/myproject/output/{file_name}.xlsx"
    
    if not os.path.isfile(path2):  
        wb.save(path2)
        print('파일 생성완료2')

    else :
        print('파일을 작업중입니다....')



파일을 작업중입니다....


In [17]:
#### 행 생성 ####

import pandas as pd
import numpy as np
from itertools import chain
# import xlsxwriter

#복제
key_lists = list(chain.from_iterable(zip(key_list,key_list))) #멀티 인덱스를 위해서 [a,b,c] => [a,a,b,b,c,c] 로 만들어줌
multi_keys = ['조회수','서브키워드'] * len(key_list)

# 조회수, 서브 키워드 입력
def hit_sub(i, time) :
    df1.loc[time][(i,'조회수')] = sum_search[i]
    df1.loc[time][(i,'서브키워드')] = sub_key[i]
    
# generate time series index
time = nowDatetime    #현재 시간 설정

df1 = pd.DataFrame(columns = [key_lists, multi_keys],index = [time])

for i in sum_search : 
    hit_sub(i, time)
    
#### 다음 행에 추가 ####
df = pd.read_excel(path2, header=[0,1], index_col=[0] )   #number 해결
df = df.append(df1, sort=False)    #컬럼 수가 다를 때 NaN 값을 넣고 행 추가

# ##### 열 너비 조정 ####
# writer = pd.ExcelWriter(path2, engine='xlsxwriter')
# df.to_excel(writer, sheet_name="Sheet1")
# workbook = writer.book
# worksheet = writer.sheets["Sheet1"]
# worksheet.set_column('A:A', 20)
# writer.save()
# print('행 추가 완료')

In [ ]:
#### 상위 5개 키워드 ####

df = pd.read_excel(path2, header=[0,1], index_col=[0] )
df_col = list(df.columns.levels[0])

hit_top5 = []

for i in df_col :
    hit_max = df[i,'조회수'].max()
    hit_top5.append(hit_max)

hit_dict = dict(zip(df_col,hit_top5)) #딕셔너리 값으로 저장

keyword_top5 = sorted(hit_dict, key=hit_dict.get, reverse = True)[:5] #탑 5개 keyword 추출
print(keyword_top5)

NameError: name 'pd' is not defined

In [ ]:
#### 19시에 코드 보내기

import matplotlib.pyplot as plt
import seaborn as sns

title_font = {'fontsize': 16, 'fontweight': 'bold'}
plt.rc('font',family='Malgun Gothic')
plt.figure(figsize=[20,10])
plt.style.use('ggplot')
plt.show(block=False)
plt.pause(1)
plt.close()



출처: https://theonly1.tistory.com/2470 [Be the only one, not the best one]

ymd = today.strftime('%Y-%m-%d') 
hms = '19:00:00'

if hms == '19:00:00':
    path_png = "C:/Workspace/project2_final/output/graphs"
    if not os.path.isdir(path_png):                                                           
        os.mkdir(path_png)
    
    path_date = f"C:/Workspace/project2_final/output/graphs/{ymd}"
    if not os.path.isdir(path_date):                                                           
        os.mkdir(path_date)    
    
    ##### 그래프 조회수 변동 ######
    for i in keyword_top5 :
        plt.title (f"키워드 : '{i}' 조회수 변동", fontsize=20)
        df.index,df[i,'조회수'].plot( kind='bar')
        #    plt.bar(df.index,df[i,'조회수'])
        #plt.show()
        plt.savefig(f'{path_date}/조회수변동-({i}).png', bbox_inches='tight')
        plt.close()
                 #.plot( kind='bar')

    ##### 그래프 키워드 관심도 #####
    for i in keyword_top5 :
        text =f'{i} 키워드 관심도'
        sub_keys = df[i].groupby(['서브키워드'])['조회수'].mean().sort_values()
        plt.title(text, fontdict=title_font, loc='center', pad= 20)
        sub_keys.plot(kind='pie', autopct = '%1.1f%%', shadow = True, startangle=110 )
        #plt.show()

        plt.savefig(f'{path_date}/관심도-({i}).png')
        plt.close()
    
    print('그래프 시각화 완료')
    
 
    


<Figure size 1440x720 with 0 Axes>

그래프 시각화 완료
